In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from finvizfinance.screener.overview import Overview
from scipy.stats import linregress
import time

In [2]:
# ==========================================
# 1.(Finviz Screening) - UPDATED with Canada
# ==========================================
def get_filtered_picks(limit_per_country=5000):
    print("--- STEP 1: Fetching Universe from Finviz (USA + Canada) ---")
    
    # Define distinct criteria for each market
    # USA = High Volume (>2M), Canada = Medium Volume (>200K)
    targets = [
        {'country': 'USA', 'vol': 'Over 2M'},
        {'country': 'Canada', 'vol': 'Over 200K'}
    ]
    
    combined_frames = []

    for target in targets:
        c_name = target['country']
        vol_req = target['vol']
        print(f"   Querying {c_name} Market (Vol: {vol_req})...")
        
        try:
            foverview = Overview()
            
            # Common Filters applied to both, with dynamic Country/Volume
            filters_dict = {
                'Country': c_name,
                'Average Volume': vol_req,
                'Market Cap.': '+Small (over $300mln)',
                
                # 1. Institutional Floor (Removes Penny Stocks)
                'Price': 'Over $5',
        
                # 2. Legitimacy Check (Ensures Options Market exists)
                'Option/Short': 'Optionable',
        
                # 3. Solvency Check (Basic Liquidity)
                'Current Ratio': 'Over 1',
        
                # 4. Profitability Filter
                'Operating Margin': 'Positive (>0%)',
            }
            
            foverview.set_filter(filters_dict=filters_dict)
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                print(f"      -> Found {len(df_results)} candidates in {c_name}.")
                combined_frames.append(df_results)
            else:
                print(f"      -> No results found for {c_name}.")
                
        except Exception as e:
            print(f"      Error in Finviz Step for {c_name}: {e}")

    # Combine and Clean Data
    if not combined_frames:
        return pd.DataFrame()
        
    df_final = pd.concat(combined_frames, ignore_index=True)
    
    # Clean up Column Names immediately
    if 'Analyst Recom' in df_final.columns:
        df_final.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
        
    # Ensure numeric Price for later
    df_final['Price'] = pd.to_numeric(df_final['Price'], errors='coerce')
    
    # Remove potential duplicates if dual-listed stocks appear twice
    df_final.drop_duplicates(subset=['Ticker'], inplace=True)
    
    print(f"   Success! Combined Total: {len(df_final)} candidates.")
    return df_final

In [3]:
# ==========================================
# 1B. SPECIALIZED UNIVERSE: FINANCIALS (USA + CANADA)
# ==========================================
def get_financial_universe():
    print("--- STEP 1 (Financials): Fetching Banking & Insurance Universe (USA + Canada) ---")
    
    # Same logic: High volume for USA, Lower volume for dual-listed Canadian banks
    targets = [
        {'country': 'USA', 'vol': 'Over 2M'},      
        {'country': 'Canada', 'vol': 'Over 200K'}  
    ]
    
    combined_frames = []
    
    for target in targets:
        c_name = target['country']
        print(f"   Querying {c_name} Financial Sector...")
        
        try:
            foverview = Overview()
            
            # --- DISTINCT FILTERS FOR BANKS ---
            filters_dict = {
                'Country': c_name,
                'Sector': 'Financial',             # <--- CRITICAL: ONLY FINANCIALS
                'Average Volume': target['vol'], 
                'Market Cap.': '+Small (over $300mln)',
                'Price': 'Over $5',
                'Option/Short': 'Optionable',      # Quality filter
            }
            
            foverview.set_filter(filters_dict=filters_dict)
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                print(f"      -> Found {len(df_results)} financial stocks in {c_name}.")
                combined_frames.append(df_results)
            else:
                print(f"      -> No financial results for {c_name}.")
                
        except Exception as e:
            print(f"      Error fetching {c_name}: {e}")

    # Combine data
    if not combined_frames:
        return pd.DataFrame()
        
    fins_universe = pd.concat(combined_frames, ignore_index=True)
    
    # Clean up Column Names
    if 'Analyst Recom' in fins_universe.columns:
        fins_universe.rename(columns={'Analyst Recom': 'Recom'}, inplace=True)
        
    # Ensure numeric Price
    fins_universe['Price'] = pd.to_numeric(fins_universe['Price'], errors='coerce')
    
    # Remove duplicates
    fins_universe.drop_duplicates(subset=['Ticker'], inplace=True)
    
    print(f"   TOTAL FINANCIAL UNIVERSE: {len(fins_universe)} stocks.")
    return fins_universe

In [4]:
# ==========================================
# 2. THE CREDIT MODEL (Z-Score & Margins)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score (Bankruptcy Risk)"""
    try:
        # Extract Key Metrics
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        # Z-Score Components
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        # Handle EBIT naming differences
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        # Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E Altman Z-Score coefficients
        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan
#The Margin Trend column is a custom metric we built to measure operational momentum. 
# It tells you whether a company is becoming more or less efficient at making money over time.
def get_margin_trend(financials):
    """Calculates 4-Year Gross Margin Slope"""
    try:
        # Update: Use up to 4 years (yfinance standard max)
        # min() prevents errors for companies with < 4 years of history
        limit = min(4, len(financials.columns))
        years = financials.columns[:limit]
        
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        # Slope Calculation
        if len(margins) < 2: return "N/A" # Need at least 2 points to calculate a slope
        
        margins = margins[::-1] # Flip to Chronological order (Oldest -> Newest)
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"



In [5]:

# ==========================================
# 2B. THE CREDIT MODEL (Z-Score & Margins)
# ==========================================
def get_interest_coverage(financials):
    """Calculates Interest Coverage Ratio (Solvency Check)"""
    try:
        # 1. Get EBIT (already logic for this in Z-score, but good to have standalone)
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan

        # 2. Get Interest Expense
        # Note: yfinance often lists this as a positive number in 'Interest Expense' 
        # or inside 'Net Income From Continuing Ops' section. 
        # We look for 'Interest Expense' or 'Interest Expense Non Operating'
        if 'Interest Expense' in financials.index:
            interest = financials.loc['Interest Expense'].iloc[0]
        elif 'Interest Expense Non Operating' in financials.index:
            interest = financials.loc['Interest Expense Non Operating'].iloc[0]
        else:
            # If no interest expense listed, they might be debt-free (Safe)
            return 100.0 

        # 3. Calculate Ratio
        # Interest is often negative in data, so we take absolute value
        interest = abs(interest)
        
        if interest == 0:
            return 100.0 # No debt cost = Infinite coverage
            
        coverage = ebit / interest
        return round(coverage, 2)
    except:
        return np.nan

def calculate_roic(financials, balance_sheet):
    """Calculates Return on Invested Capital (Quality Check)"""
    try:
        # 1. NOPAT (Net Operating Profit After Tax)
        # Simplified: EBIT * (1 - Tax Rate)
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        tax_provision = financials.loc['Tax Provision'].iloc[0] if 'Tax Provision' in financials.index else 0
        pre_tax_income = financials.loc['Pretax Income'].iloc[0] if 'Pretax Income' in financials.index else 0
        
        # Calculate effective tax rate, cap at 25% if data is weird, 0% if loss
        if pre_tax_income > 0:
            tax_rate = tax_provision / pre_tax_income
            tax_rate = max(0.0, min(tax_rate, 0.30)) # Normalize between 0% and 30%
        else:
            tax_rate = 0.21 # Default corporate tax rate assumption if negative earnings
            
        nopat = ebit * (1 - tax_rate)

        # 2. Invested Capital
        # Formula: Total Equity + Total Debt - Cash
        total_equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
        
        total_debt = 0
        if 'Total Debt' in balance_sheet.index:
            total_debt = balance_sheet.loc['Total Debt'].iloc[0]
        elif 'Long Term Debt' in balance_sheet.index: # Fallback
            total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
            
        cash = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
        
        invested_capital = total_equity + total_debt - cash
        
        if invested_capital <= 0:
            return np.nan
            
        roic = (nopat / invested_capital) * 100
        return round(roic, 2)
    except:
        return np.nan

In [6]:
# ==========================================
# 3. EXECUTION ENGINE (SPLIT BY TIER)
# ==========================================
import time

# --- HELPER FUNCTIONS (Keep these at the top) ---
def get_interest_coverage(financials):
    """Calculates Interest Coverage Ratio (Solvency Check)"""
    try:
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan

        if 'Interest Expense' in financials.index:
            interest = financials.loc['Interest Expense'].iloc[0]
        elif 'Interest Expense Non Operating' in financials.index:
            interest = financials.loc['Interest Expense Non Operating'].iloc[0]
        else:
            return 100.0

        interest = abs(interest)
        if interest == 0: return 100.0
            
        return round(ebit / interest, 2)
    except:
        return np.nan

def calculate_roic(financials, balance_sheet):
    """Calculates Return on Invested Capital (Quality Check)"""
    try:
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        tax_provision = financials.loc['Tax Provision'].iloc[0] if 'Tax Provision' in financials.index else 0
        pre_tax_income = financials.loc['Pretax Income'].iloc[0] if 'Pretax Income' in financials.index else 0
        
        if pre_tax_income > 0:
            tax_rate = max(0.0, min(tax_provision / pre_tax_income, 0.30))
        else:
            tax_rate = 0.21
            
        nopat = ebit * (1 - tax_rate)

        total_equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
        total_debt = balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in balance_sheet.index else 0
        if total_debt == 0 and 'Long Term Debt' in balance_sheet.index:
             total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
             
        cash = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
        
        invested_capital = total_equity + total_debt - cash
        if invested_capital <= 0: return np.nan
            
        return round((nopat / invested_capital) * 100, 2)
    except:
        return np.nan

# --- MAIN ENGINE ---
def run_screener_split():
    # A. Get the List from Step 1
    candidates = get_filtered_picks()
    
    if candidates.empty:
        print("No stocks found to analyze.")
        return None, None, None, None, None

    print(f"\n--- STEP 2: Running Credit Risk Model on {len(candidates)} Stocks ---")
    print("Separating stocks into distinct tiers...")
    
    fortress_data = []
    moonshot_data = []
    distress_data = []
    financial_data = [] 
    middle_data = []    
    
    for index, row in candidates.iterrows():
        ticker = row['Ticker']
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. SEPARATE FINANCIALS
            sector = info.get('sector', 'Unknown')
            if 'Financial' in sector:
                financial_data.append({
                    'Ticker': ticker,
                    'Sector': sector,
                    'Price': row['Price'],
                    'Recom': row.get('Recom', 'N/A')
                })
                continue
            
            # 2. FETCH DATA
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty:
                continue
            
            # 3. RUN METRICS
            z_score = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            # Data packet to save (ORDER CHANGED HERE)
            stock_data = {
                'Ticker': ticker,
                'Company': row.get('Company', 'N/A'),
                'Z-Score': z_score,
                'Price': row['Price'],       # <--- Moved to Column 4
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Recom': row.get('Recom', 'N/A'),
                'Sector': sector
            }
            
            # 4. SORT INTO LISTS
            if (z_score > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress_data.append(stock_data)
                
            elif (z_score < 1.8) or (int_cov < 1.5):
                distress_data.append(stock_data)
                
            elif (z_score < 2.5) and (trend == "Improving"):
                moonshot_data.append(stock_data)
            
            else:
                middle_data.append(stock_data)
            
            if index % 10 == 0:
                print(f"   Processed {index} / {len(candidates)}...", end='\r')

        except Exception:
            continue

    # B. Convert Lists to DataFrames
    fortress_df = pd.DataFrame(fortress_data)
    moonshot_df = pd.DataFrame(moonshot_data)
    distress_df = pd.DataFrame(distress_data)
    financial_df = pd.DataFrame(financial_data)
    middle_df = pd.DataFrame(middle_data)
    
    print(f"\n\n--- ANALYSIS COMPLETE ---")
    print(f"Fortress (Safe):     {len(fortress_df)} stocks")
    print(f"Moonshot (Risky):    {len(moonshot_df)} stocks")
    print(f"Distress (Avoid):    {len(distress_df)} stocks")
    print(f"Financials (Skip):   {len(financial_df)} stocks")
    print(f"Middle of Road:      {len(middle_df)} stocks")
    
    return fortress_df, moonshot_df, distress_df, financial_df, middle_df

# --- RUN IT ---
fortress_df, moonshot_df, distress_df, fins_df, mid_df = run_screener_split()

--- STEP 1: Fetching Universe from Finviz (USA + Canada) ---
   Querying USA Market (Vol: Over 2M)...
No ticker found.
      Error in Finviz Step for USA: 'NoneType' object has no attribute 'empty'
   Querying Canada Market (Vol: Over 200K)...
No ticker found.
      Error in Finviz Step for Canada: 'NoneType' object has no attribute 'empty'
No stocks found to analyze.


In [7]:
if fortress_df is not None and not fortress_df.empty:
    print("--- FORTRESS STOCKS (High Quality) ---")
    
    display(fortress_df.sort_values(by='Price', ascending=True))


In [8]:
#high risk picks
if moonshot_df is not None and not moonshot_df.empty:
    print("--- MOONSHOT STOCKS (Speculative) ---")
    display(moonshot_df.sort_values(by='Price', ascending=True))

In [9]:
#very high risk / avoid picks
if distress_df is not None and not distress_df.empty:
    print("--- DISTRESS (Avoid / Short Candidates) ---")
    display(distress_df.sort_values(by='Z-Score', ascending=True))

In [10]:
#mid picks for analysis
mid_df
if mid_df is not None and not mid_df.empty:
    print("--- Mid Picks requires more analysis ---")
    display(mid_df.sort_values(by='Z-Score', ascending=True))

In [11]:
# ==========================================
# 4. SPECIALIZED FINANCIALS ENGINE (FIXED)
# ==========================================
def run_financials_analysis(df):
    """
    Takes the raw list of Financial stocks (Banks/Insurance),
    fetches valuation metrics, and filters for Value & Income.
    """
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return

    print(f"--- ANALYZING {len(df)} FINANCIAL STOCKS (USA + TSX) ---")
    print("Fetching P/E, P/B, Dividend, and Analyst Ratings...\n")
    
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        try:
            # Fetch Data
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. VALUATION METRICS
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            
            # 2. EFFICIENCY & INCOME
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            # 3. ANALYST RATING (The Fix)
            # We fetch directly from YF to ensure we get a number (1=Strong Buy, 5=Sell)
            recom = info.get('recommendationMean', None)
            
            # Fallback: If no analyst covers it, assume 3.0 (Hold) so it doesn't crash
            if recom is None: 
                recom = 3.0

            # 4. GROWTH
            rev_growth = info.get('revenueGrowth', np.nan)

            # Save it
            bank_data.append({
                'Ticker': ticker,
                'Company': row.get('Company', ticker),
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield * 100, 2),
                'Recom': float(recom),  # Ensure it is a float for filtering
                'Rev_Growth': rev_growth,
                'Sector': row.get('Sector', 'Financial')
            })
            
        except Exception:
            continue
            
    # Create the Enriched DataFrame
    rich_fins_df = pd.DataFrame(bank_data)
    
    if rich_fins_df.empty:
        print("Could not fetch details for financial stocks.")
        return

    # ==========================================
    # FILTER 1: "UNDERVALUED BANKING" (The Value Play)
    # Criteria: Cheap (P/E < 15), Trading near assets (P/B < 1.2), Profitable (ROE > 8%)
    # ==========================================
    value_picks = rich_fins_df[
        (rich_fins_df['P/E'] < 15) & 
        (rich_fins_df['P/B'] < 1.2) & 
        (rich_fins_df['ROE'] > 0.08)
    ].copy()
    
    print(f"--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---")
    if not value_picks.empty:
        display(value_picks.sort_values(by='P/B', ascending=True))
    else:
        print("No stocks met the strict value criteria.")

    # ==========================================
    # FILTER 2: "INCOME COMPOUNDERS" (The Dividend Play)
    # Criteria: High Yield (>2.5%), Growing (>0%), Sustainable P/E (<20)
    # PLUS: Analyst Buy Rating (Score <= 2.5)
    # ==========================================
    income_picks = rich_fins_df[
        (rich_fins_df['Yield%'] >= 2.5) & 
        (rich_fins_df['P/E'] < 20) &
        (rich_fins_df['ROE'] > 0.05) &
        (rich_fins_df['Recom'] <= 2.0)  #consider removing or changing later
    ].copy()

    print(f"\n--- FILTER 2: INCOME GENERATORS (Yield > 2.5% & Buy Rating) ---")
    if not income_picks.empty:
        display(income_picks.sort_values(by='Yield%', ascending=False))
    else:
        print("No stocks met the dividend criteria.")

    return rich_fins_df

In [12]:
# ==========================================
# MAIN EXECUTION: FINANCIALS TRACK
# ==========================================

# 1. Fetch the data (Uses the function from Part 1B)
# Note: This checks USA (>2M Volume) and Canada (>200K Volume)
fins_df = get_financial_universe()

# 2. Analyze the data (Uses the function from Part 4)
# This will display the tables for "Undervalued Banks" and "Income Generators"
if not fins_df.empty:
    final_banks_df = run_financials_analysis(fins_df)
else:
    print("No financial stocks found to analyze.")

--- STEP 1 (Financials): Fetching Banking & Insurance Universe (USA + Canada) ---
   Querying USA Financial Sector...
No ticker found.
      Error fetching USA: 'NoneType' object has no attribute 'empty'
   Querying Canada Financial Sector...
No ticker found.
      Error fetching Canada: 'NoneType' object has no attribute 'empty'
No financial stocks found to analyze.


In [13]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 11 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...


C:\Users\jdcc3\AppData\Local\Temp\ipykernel_7948\4051987151.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,13.49,-1.10,17.68,-46.32,1.40,21.75,0.67,3.18
0,ADMA,19.28,-1.33,17.90,-24.89,1.00,30.00,0.41,3.26
9,SEI,44.65,-0.93,32.46,-21.72,1.17,65.45,0.31,5.90
7,MIR,23.75,-0.92,19.89,-21.56,1.12,30.62,0.34,3.49
10,UBER,81.26,0.14,84.45,-20.33,1.47,112.40,0.33,2.41
5,FLEX,63.33,-0.35,48.21,-12.31,1.45,76.00,0.23,2.90
2,ARCC,20.20,1.00,20.48,-9.77,1.27,22.64,0.85,1.38
8,ONB,22.98,-0.35,21.40,-3.76,1.85,25.92,0.70,2.14
1,APG,39.57,0.43,31.34,-2.49,1.45,43.40,0.29,1.92
3,BANC,19.70,0.15,15.35,-1.84,1.27,22.14,0.44,2.18
